# **SimTS** training

## Libraries

In [ ]:
import os
import math
import time
import torch
import random
import datetime

import numpy as np

import forecasting as fcst

from simts import SimTS
from data_loader import *
from utils import pkl_save, save_checkpoint_callback

## Parameters

Set accurately the following parameters to train a SimTS model.

In [ ]:
dataset = "ETTh1" # The dataset name.
dir = os.getcwd() # The folder name used to save model outputs and evaluation metrics (current directory)
loader = "univar" # The data loader used to load the data. This can be set to 'multivar' or 'univar'.
batch_size = 8    # The batch size.
K = 201           # K in the paper. Length of history segmentation.
lr = 0.001        # Learning rate.
repr_dims = 320   # Latent space dimensions.
T = 402           # Training sample in the paper.
iters = None      # Number of iterations.
epochs = 5        # Number of epochs
eval = True       # Whether to perform evaluation after training.
save_every = 5    # Save the checkpoints every <save_every> epochs/iterations.
seed = 1499       # Set random seed.

kernel_list = [2 ** i for i in range(math.floor(math.log2(K)) + 1)] # List of kernel sizes used.

In [ ]:
# Store parameters inside a dictionary.
args = dict(
    dataset=dataset,
    dir=dir,
    loader=loader,
    batch_size=batch_size,
    K=K,
    lr=lr,
    repr_dims=repr_dims,
    T=T,
    kernel_list=kernel_list,
    iters=iters,
    epochs=epochs,
    eval=eval,
    save_every=save_every,
    seed=seed,
)

In [ ]:
print("Dataset:", dataset)
print("Arguments:", str(args))

In [ ]:
# Set random seeds.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# Set torch device.
if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device = "cpu"

In [ ]:
# Model configuration parameters.
config = dict(
    batch_size=batch_size,
    lr=lr,
    kernel_list=kernel_list,
    output_dims=repr_dims,
    K=K,
    T=T,
)

In [ ]:
# Set checkpoint callbacks.
if save_every is not None:
    unit = "epoch" if epochs is not None else "iter"
    config[f"after_{unit}_callback"] = save_checkpoint_callback(save_every, unit)

run_dir = dir + "\\" + dataset
os.makedirs(run_dir, exist_ok=True)

## Data loading

Load the chosen data with the appropriate data loader.

In [ ]:
print("Loading data..", end="")

if loader == "multivar":
    task_type = "forecasting"
    data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = load_forecast_csv(dataset, univar=False)
    train_data = data[:, train_slice]

elif loader == "univar":
    task_type = "forecasting"
    data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = load_forecast_csv(dataset)
    train_data = data[:, train_slice]
    
else:
    raise ValueError(f"Unknown loader {loader}")

print("Done.", train_data.shape)

## Training

Train the SimTS model.

In [ ]:
t = time.time()

print("Train data size:", train_data.shape)

model = SimTS(
    input_dims=train_data.shape[1],
    **config
)

loss_log, best_net = model.fit(
    train_data,
    n_epochs=epochs,
    n_iters=iters,
    verbose=True
)

t = time.time() - t
print(f"Training time: {datetime.timedelta(seconds=t)}")

## Evaluation

Evaluate the output results.

In [ ]:
if eval:
    out, eval_res = fcst.eval_forecasting(model, data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols)
    
    pkl_save(f"{run_dir}/out.pkl", out)
    pkl_save(f"{run_dir}/eval_res.pkl", eval_res)
    print("Evaluation result:", eval_res)

print("Finished.")